## 1 - Importação de Bibliotecas

In [0]:
import pandas as pd
import urllib.request
import json
import zipfile
import io
from pyspark.sql.functions import col, coalesce, round, when, avg, count, date_format, to_date, desc, abs, format_number

## 2 - Carga dos dados

In [0]:
url = "https://drive.google.com/u/0/uc?id=14nMMC19vpQ8s69TqCkQZh7SxJxlpaOzW&export=download&confirm=t&uuid=1502da0c-6c26-4912-9be9-70462bb36649&at=AKKF8vxyDAoKgvLHnb-SP4tC-MeN:1683068739477"
response = urllib.request.urlopen(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.read()))

list_json = []
for filename in zip_file.namelist():
    with zip_file.open(filename) as file:
        json_data = file.read()
        try:
          json_data = json.loads(json_data)
          list_json.append(json_data)
        except BaseException as e:
          print('The file contains invalid JSON')

The file contains invalid JSON
The file contains invalid JSON


## 3 - Criação do dataframe

In [0]:
pdf = pd.json_normalize(list_json, record_path='assortment')
df_ascential = spark.createDataFrame(pdf)

## 4 - Tratamento dos dados

In [0]:
df_ascential = df_ascential.withColumn('retailerPrice', when(col('retailerPrice').isNull(), 0).otherwise(col('retailerPrice'))) \
                         .withColumn('manufacturerPrice', when(col('manufacturerPrice').isNull(), 0).otherwise(col('manufacturerPrice'))) \
                         .withColumn('priceVariation', when(col('priceVariation').isNull(), 0).otherwise(col('priceVariation')))

## 5 - Variação Percentual do preço entre varejista e fabricante

In [0]:
df_selected = df_ascential.select('idRetailerSKU', 'manufacturerPrice', 'retailerPrice')

df_avg = df_selected.groupBy('idRetailerSKU') \
                    .agg(avg('manufacturerPrice').alias('avg_manufacturerPrice'),
                         avg('retailerPrice').alias('avg_retailerPrice'))

df_variation = df_avg.withColumn('variation_percentage', ((col('avg_retailerPrice') - col('avg_manufacturerPrice')) / col('avg_manufacturerPrice')) * 100)

df_top10 = df_variation.orderBy(col('variation_percentage').desc()).limit(10)
df_top10 = df_top10.withColumn("variation_percentage", round(col("variation_percentage"),2))
df_top10 = df_top10.select('idRetailerSKU', 'variation_percentage')

df_top10.show()

+-------------+--------------------+
|idRetailerSKU|variation_percentage|
+-------------+--------------------+
|      1925529|           258882.17|
|        64352|            249899.0|
|      1980360|           238398.87|
|      1980361|            179978.3|
|       722371|            179800.0|
|        64319|            169899.0|
|      1980362|            124221.0|
|       488054|            12595.75|
|       487755|             10899.0|
|       744562|            10592.22|
+-------------+--------------------+



## 6 - variação do preço

In [0]:
top_10_variations = df_ascential.groupBy('idRetailerSKU') \
                      .agg({'priceVariation': 'max'}) \
                      .orderBy(desc('max(priceVariation)')) \
                      .limit(10)
top_10_variations = top_10_variations.withColumn('max(priceVariation)', format_number(col('max(priceVariation)'), 2))
top_10_variations = top_10_variations.withColumnRenamed('max(priceVariation)', 'price_variation')
top_10_variations.show()

+-------------+---------------+
|idRetailerSKU|price_variation|
+-------------+---------------+
|      1925529|     284,400.00|
|        64352|     249,899.00|
|      1980361|     240,911.00|
|      1980360|     239,899.00|
|       722371|     179,800.00|
|        64319|     169,899.00|
|      1980362|     129,899.00|
|       488054|      12,598.35|
|       744822|      11,699.00|
|       744562|      11,034.00|
+-------------+---------------+



## 7 - Os 10 produtos que apresentam maior indisponibilidade

In [0]:
df_filtered = df_ascential.filter(col('available') == False)

df_formatted = df_filtered.withColumn('dateTimeReference_formatted',
                                       date_format(to_date(col('dateTimeReference'), "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'")
                                      )

df_count = df_formatted.groupBy('idRetailerSKU').agg(count('*').alias('count'))

df_max_unavailability = df_count.orderBy(col('count').desc()).limit(10)

df_max_unavailability.show()

+-------------+-----+
|idRetailerSKU|count|
+-------------+-----+
|       498641|   29|
|      1089181|   29|
|       747600|   29|
|      1008771|   29|
|       491942|   29|
|       603194|   29|
|      1008612|   29|
|      1922968|   29|
|      1957927|   29|
|      1892300|   29|
+-------------+-----+



## 8 - Produtos que estão sendo vendidos com valores muito distantes do ideal

In [0]:
df_produtos_diff = df_ascential.select(col('idRetailerSKU'), col('retailerPrice'), col('manufacturerPrice'))
df_produtos_diff = df_produtos_diff.withColumn('diferenca_preco', col('retailerPrice') - col('manufacturerPrice'))
df_max_diff = df_produtos_diff.groupBy('idRetailerSKU').agg({'diferenca_preco': 'max'}).orderBy('max(diferenca_preco)', ascending=False)
df_max_diff = df_max_diff.withColumnRenamed('max(diferenca_preco)', 'diferenca_preco')
df_max_diff = df_max_diff.limit(10)
df_max_diff.show()

+-------------+---------------+
|idRetailerSKU|diferenca_preco|
+-------------+---------------+
|      1907007|        4924.28|
|      1907784|        3729.99|
|      1907680|        3549.99|
|      1925529|         2844.0|
|        64352|        2498.99|
|      1980361|        2409.11|
|      1907455|        2398.99|
|      1980360|        2398.99|
|      1905003|        2327.51|
|      1905620|        1999.99|
+-------------+---------------+



In [0]:
display(df_ascential.select(col('idRetailerSKU'), col('retailerPrice'), col('manufacturerPrice')).filter(col('idRetailerSKU') == 1907007))

idRetailerSKU,retailerPrice,manufacturerPrice
1907007,0.0,0.0
1907007,0.0,0.0
1907007,4822.75,0.0
1907007,0.0,0.0
1907007,0.0,0.0
1907007,4924.28,0.0
1907007,4924.28,0.0
1907007,4924.28,0.0
1907007,0.0,0.0
1907007,4822.75,0.0


In [0]:
display(df_ascential.filter(col('idRetailerSKU') == 498641 ))

dateTimeReference,idRetailerSKU,modifiedDate,seller,retailerFinalUrl,retailerProductCode,available,unavailable,notListed,screenshot,retailerDescription,retailerPrice,retailerTitle,retailerRatingCount,retailerFirstImageType,heroImageFlag,manufacturerExtraImagesTotal,manufacturerImagesTotal,retailerExtraImagesTotal,retailerImagesTotal,retailerDescriptionFoundWords,descriptionFlag,descriptionPercentage,manufacturerDescription,manufacturerPrice,priceInsideRange,priceAboveRange,priceBelowRange,priceOutOfThePolicy,priceVariation,useManufacturerPriceVariationRule,useProductPriceRangeRule,retailerTitleFoundWords,titleFlag,titlePercentage,brandRetailerTitle,manufacturerTitle,retailerAverageRating,retailerReviewCount,correctExtraImagesTotal,extraImagesPercentage,primaryImageFlag,retailerFromPrice,priceDiscount
2022-12-02T00:00:00.000Z,498641,2022-12-02T00:16:45.247Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-22T00:00:00.000Z,498641,2022-12-22T00:19:31.689Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-13T00:00:00.000Z,498641,2022-12-13T00:21:05.898Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-09T00:00:00.000Z,498641,2022-12-09T00:19:50.306Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-15T00:00:00.000Z,498641,2022-12-15T00:19:20.845Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-28T00:00:00.000Z,498641,2022-12-28T00:19:34.349Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-14T00:00:00.000Z,498641,2022-12-14T00:19:36.454Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-06T00:00:00.000Z,498641,2022-12-06T00:16:29.797Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-07T00:00:00.000Z,498641,2022-12-07T00:19:50.686Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-12-08T00:00:00.000Z,498641,2022-12-08T00:19:44.017Z,Americanas,null,null,false,false,true,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
